In [4]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv
import cProfile

In [2]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [3]:
#definisco una matrice k x (n-k)
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [4]:
max_column = 11

In [5]:
#number of iteration for Montecarlo simulatiom
num_iterations = 1000

In [20]:
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

#initial matrix x matrix with s or s^(-1)
def make_matrix_xS(matrix, field_order):
    #start = time.perf_counter() this is for total time
    matrix_S = field(np.zeros((k, k), dtype = int))
    for i, row in enumerate(matrix):
        if np.all(row == row[0]):
            if row[0] != 0:
                row[:] = 1
                matrix_S[i,i] = 1
            else:
                matrix_S[i, i] = 1
        else:
            #print("iter:", i)
            #print("Type row: ", type(row)) IS GALOIS (CONFIRMED)
            s = np.sum(row)
            #print(s)
            s_prime = np.sum(row ** (field_order-2))
            #print(s_prime)
            if s != 0:
                matrix_S[i,i] = np.reciprocal(s)
            elif s_prime != 0:
                matrix_S[i,i] = s_prime
            else:
                #end = time.perf_counter() this is for total time
                #duration = (end - start) * 1000 this is for total time
                print("esco")
                return []#, duration this is for total time
    #end = time.perf_counter() this is for total time
    #duration = (end - start) * 1000 this is for total time
    #print("Matrix_S:", matrix_S)
    matrix = matrix_S @ matrix
    #print(type(matrix), matrix)
    return matrix#, duration this is for total time

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter):  
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, [], 0, 0, error_counter
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    #start = time.perf_counter()
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration = (end - start) * 1000
    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration = (end - start) * 1000    
    
    start = time.perf_counter() #this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts1 = np.zeros((matrix.shape[0], max_column), dtype = int)
    for i, row in enumerate(matrix):
        #BY REMOVING THIS FILTERING (BELOW) WHICH WOULD ADD OVERHEAD AS I HAVE TO SCROLL THROUGH THE ENTIRE VECTOR REGARDLESS
        #I WILL SAVE IN TERMS OF TIME
        #valid_values = row[row < max_column]
        vector_of_counts1[i] = np.bincount(row, minlength = max_column)[:max_column]
    #print(vector_of_counts1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration1 = (end - start) * 1000
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    """for raw in matrix[sorted_index1]:
        print(np.sort(raw))"""
    #print()
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns

def algorithm_test(matrix1, matrix2):
    row1_sort, duration1 = order_row(matrix1, 0)
    #print(row1_sort)
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)

    row2_sort, duration2 = order_row(matrix2, 0)
    #print(row2_sort)
    matrix2_sorted = sort_by_column(row2_sort, range(k), 0)
    #print(matrix2_sorted)

    return np.array_equal(matrix1_sorted, matrix2_sorted), duration1, duration2

def algorithm_test1(matrix1, error_counter):
    row1_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row(matrix1, error_counter)
    #print(row1_sort)
    if len(row1_sort) == 0:
        return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)
    return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter

def montecarlo_simulation(num_iterations):
    for i in range(num_iterations):
        A = field(np.random.randint(0, field.order, size=(k, n_k))) 
        perm1 = make_bin_matrix((k, n_k), field)
        perm2 = make_bin_matrix((k, n_k), field)
        matrix_diagonal = np.zeros((k, n_k))
        #replace the values ​​on the diagonal and min(k_n_k) tells me the size of the diagonal equal to the minimum of the two
        matrix_diagonal = field(np.fill_diagonal(matrix_diagonal, np.random.randint(0, field.order, min(k, n_k))))
        A_primo = field(perm1 @ matrix_diagonal @ A @ perm2)
        check, duration1, duration2 = algorithm_test(A, A_primo)
        if check:
            print(i)



def motecarlo_simulation1(num_iterations):
    vector_operations = []
    vector_operations1 = []
    vector_duration1 = []
    vector_duration1_2 = []
    error_counter = 0
    duration1_accumulator = 0
    duration1_2_accumulator = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    #this is for total time
    #csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_terza_revisione/dati_tot_time.csv'
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_terza_revisione/dati.csv'
    file_exists = os.path.exists(csv_file_path)
    #this is for total time
    #with open('dati_tot_time.csv', mode='a', newline='') as file:
    with open('dati.csv', mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns'])

        for i in range(num_iterations):
            A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            while(has_duplicate_rows(A)):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            
            #this is for total time
            #A, duration_diag = make_matrix_xS(A, field.order)
            A = make_matrix_xS(A, field.order)
            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = algorithm_test1(A, error_counter)
            if check:
                duration1_accumulator += duration1_1
                duration1_2_accumulator += durations1_2
                operations_accumulator += operations
                operations1_accumulator += operations1
                vector_operations.append(operations)
                vector_operations1.append(operations1)
                vector_duration1.append(duration1_1)
                vector_duration1_2.append(durations1_2)
                #this is for total time
                #writer_csv.writerow([5,i, max_column, (duration1_1 + duration_diag), (durations1_2 + duration_diag), operations, operations1])
                writer_csv.writerow([5,i, max_column, duration1_1, durations1_2, operations, operations1])

    #there i do a shapiro test for check if data follow a normal distribution
    stat, p_value_norm = stats.shapiro(vector_duration1)
    print(f"Statistic: {stat}, p-value: {p_value_norm}")
    if p_value_norm > 0.05:
        print("Non si può rifiutare H0: i dati sono normali.")
    else:
        print("H0 rifiutata: i dati NON sono normali.")

    #this test is the t test for non normal distribution data --> if p_value < 0.05 --> reject H0 --> statistically
    #significant difference between the two data sets compared
    stat_non_norm, p_value_non_norm = stats.wilcoxon(vector_duration1, vector_duration1_2)
    print(f"Statistics: {stat_non_norm}, p_value: {p_value_non_norm}")

    t_stat, p_value = stats.ttest_rel(vector_operations, vector_operations1)
    print(f"T-statistica: {t_stat}, P-value: {p_value}")
    alpha = 0.05
    if p_value < alpha:
        print("La differenza tra operazioni normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")

    t_stat1, p_value1 = stats.ttest_rel(vector_duration1, vector_duration1_2)
    print(f"T-statistica: {t_stat1}, P-value: {p_value1}")
    if p_value1 < alpha:
        print("La differenza tra durate normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")
            
    return duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value, (num_iterations-error_counter)/num_iterations, p_value1


In [25]:
cProfile.run('motecarlo_simulation1(500)', sort='cumulative')

True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positi

In [11]:
duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value, good_perc, p_value1 = motecarlo_simulation1(num_iterations)

True
If positivo
True
If positivo


True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positi

In [8]:
#print some stats
#this is for total time
#nome_file = "stat_without_sort_with_S_tot_time.txt"
nome_file = "stat_without_sort_with_S.txt"


with open(nome_file, 'a') as file:
    file.write(f"Qui c'è il limite colonne a: {max_column} file 5\n")
    file.write(f"Tempo esecuzione con tutte colonne: {duration1_accumulator} ms\n")
    file.write(f"Tempo esecuzione con {max_column} colonne: {duration1_2_accumulator} ms\n")
    file.write(f"Numero operazioni con tutte colonne: {operations_accumulator}\n")
    file.write(f"Numero operazioni con {max_column} colonne: {operations1_accumulator}\n")
    file.write(f"P-value per operazioni: {p_value}\n")
    file.write(f"P-value per tempi: {p_value1}\n")
    file.write(f"Percentuale test positivi (tutte colonne = meno colonne): {good_perc}%\n\n")